In [2]:
# PERT Implementation with Critical Path
tasks = {
    'A': {'time': (1, 2, 3), 'deps': []},
    'B': {'time': (2, 4, 6), 'deps': ['A']},
    'C': {'time': (1, 1, 1), 'deps': ['A']},
    'D': {'time': (3, 5, 7), 'deps': ['B', 'C']}
}

# Step 1: Expected time
for t in tasks:
    o, m, p = tasks[t]['time']
    tasks[t]['ET'] = (o + 4*m + p) / 6

# Step 2: Topological order
order, visited = [], set()
def dfs(n):
    if n in visited: return
    visited.add(n)
    for d in tasks[n]['deps']: dfs(d)
    order.append(n)
for t in tasks: dfs(t)

# Step 3: Forward pass
ES, EF = {}, {}
for t in order:
    ES[t] = max([EF.get(d, 0) for d in tasks[t]['deps']]) if tasks[t]['deps'] else 0
    EF[t] = ES[t] + tasks[t]['ET']

# Step 4: Backward pass
LF, LS = {}, {}
for t in reversed(order):
    LF[t] = min([LS.get(n, EF[t]) for n in tasks if t in tasks[n]['deps']]) if any(t in tasks[n]['deps'] for n in tasks) else EF[t]
    LS[t] = LF[t] - tasks[t]['ET']

# Step 5: Print and identify critical path
print("PERT Chart:\nTask | ET | ES | EF | LS | LF | Slack")
critical = []
for t in order:
    slack = LS[t] - ES[t]
    if slack == 0: critical.append(t)
    print(f" {t}   |{tasks[t]['ET']:.1f} | {ES[t]:.1f} | {EF[t]:.1f} | {LS[t]:.1f} | {LF[t]:.1f} |  {slack:.1f}")

print("\nCritical Path:", " → ".join(critical))


PERT Chart:
Task | ET | ES | EF | LS | LF | Slack
 A   |2.0 | 0.0 | 2.0 | 0.0 | 2.0 |  0.0
 B   |4.0 | 2.0 | 6.0 | 2.0 | 6.0 |  0.0
 C   |1.0 | 2.0 | 3.0 | 5.0 | 6.0 |  3.0
 D   |5.0 | 6.0 | 11.0 | 6.0 | 11.0 |  0.0

Critical Path: A → B → D


In [3]:
# CPM with Critical Path Detection
tasks = {
    'A': {'duration': 2, 'deps': []},
    'B': {'duration': 4, 'deps': ['A']},
    'C': {'duration': 1, 'deps': ['A']},
    'D': {'duration': 5, 'deps': ['B', 'C']}
}

# Topological sort
order, visited = [], set()
def dfs(n):
    if n in visited: return
    visited.add(n)
    for d in tasks[n]['deps']: dfs(d)
    order.append(n)
for t in tasks: dfs(t)

# Forward pass
ES, EF = {}, {}
for t in order:
    ES[t] = max([EF.get(d, 0) for d in tasks[t]['deps']]) if tasks[t]['deps'] else 0
    EF[t] = ES[t] + tasks[t]['duration']

# Backward pass
LF, LS = {}, {}
for t in reversed(order):
    LF[t] = min([LS.get(n, EF[t]) for n in tasks if t in tasks[n]['deps']]) if any(t in tasks[n]['deps'] for n in tasks) else EF[t]
    LS[t] = LF[t] - tasks[t]['duration']

# Output and critical path
print("CPM Chart:\nTask | Dur | ES | EF | LS | LF | Slack")
critical = []
for t in order:
    slack = LS[t] - ES[t]
    if slack == 0: critical.append(t)
    print(f" {t}   |  {tasks[t]['duration']}  | {ES[t]}  | {EF[t]}  | {LS[t]}  | {LF[t]}  |   {slack}")

print("\nCritical Path:", " → ".join(critical))


CPM Chart:
Task | Dur | ES | EF | LS | LF | Slack
 A   |  2  | 0  | 2  | 0  | 2  |   0
 B   |  4  | 2  | 6  | 2  | 6  |   0
 C   |  1  | 2  | 3  | 5  | 6  |   3
 D   |  5  | 6  | 11  | 6  | 11  |   0

Critical Path: A → B → D


##  Combined PERT & CPM with Critical Path

In [6]:
# Combined PERT and CPM Tool

# ----- PERT Section -----
print("PERT Analysis")
pert_tasks = {
    'A': {'time': (1, 2, 3), 'deps': []},
    'B': {'time': (2, 4, 6), 'deps': ['A']},
    'C': {'time': (1, 1, 1), 'deps': ['A']},
    'D': {'time': (3, 5, 7), 'deps': ['B', 'C']}
}

for t in pert_tasks:
    o, m, p = pert_tasks[t]['time']
    ET = (o + 4*m + p) / 6
    VAR = ((p - o) / 6) ** 2
    pert_tasks[t].update({'ET': ET, 'VAR': VAR, 'SD': VAR**0.5})

# Topological sort
order, visited = [], set()
def dfs(n, graph):
    if n in visited: return
    visited.add(n)
    for d in graph[n]['deps']: dfs(d, graph)
    order.append(n)
for t in pert_tasks: dfs(t, pert_tasks)

# Forward pass for total expected time
ET_total = {}
for t in order:
    ET_total[t] = max([ET_total.get(d, 0) for d in pert_tasks[t]['deps']] + [0]) + pert_tasks[t]['ET']

# Backtrack to find critical path
max_time = max(ET_total.values())
critical_path = []
for t in reversed(order):
    if ET_total[t] == max_time:
        while t:
            critical_path.insert(0, t)
            preds = [d for d in pert_tasks[t]['deps'] if ET_total[t] - pert_tasks[t]['ET'] == ET_total[d]]
            t = preds[0] if preds else None
        break

print("Task | ET  | Variance | Std Dev")
for t in order:
    print(f" {t}   | {pert_tasks[t]['ET']:.2f} |  {pert_tasks[t]['VAR']:.4f} |   {pert_tasks[t]['SD']:.2f}")
print("PERT Critical Path:", " → ".join(critical_path))

# ----- CPM Section -----
print("\nCPM Analysis")
cpm_tasks = {
    'A': {'duration': 2, 'deps': []},
    'B': {'duration': 4, 'deps': ['A']},
    'C': {'duration': 1, 'deps': ['A']},
    'D': {'duration': 5, 'deps': ['B', 'C']}
}

order, visited = [], set()
for t in cpm_tasks: dfs(t, cpm_tasks)

ES, EF = {}, {}
for t in order:
    ES[t] = max([EF.get(d, 0) for d in cpm_tasks[t]['deps']] + [0])
    EF[t] = ES[t] + cpm_tasks[t]['duration']

LF, LS = {}, {}
for t in reversed(order):
    LF[t] = min([LS.get(n, EF[t]) for n in cpm_tasks if t in cpm_tasks[n]['deps']] + [EF[t]])
    LS[t] = LF[t] - cpm_tasks[t]['duration']

critical_cpm = [t for t in order if LS[t] == ES[t]]

print("Task | Dur | ES | EF | LS | LF | Slack")
for t in order:
    slack = LS[t] - ES[t]
    print(f" {t}   | {cpm_tasks[t]['duration']}  | {ES[t]}  | {EF[t]}  | {LS[t]}  | {LF[t]}  |  {slack}")
print("CPM Critical Path:", " → ".join(critical_cpm))


PERT Analysis
Task | ET  | Variance | Std Dev
 A   | 2.00 |  0.1111 |   0.33
 B   | 4.00 |  0.4444 |   0.67
 C   | 1.00 |  0.0000 |   0.00
 D   | 5.00 |  0.4444 |   0.67
PERT Critical Path: A → B → D

CPM Analysis
Task | Dur | ES | EF | LS | LF | Slack
 A   | 2  | 0  | 2  | 0  | 2  |  0
 B   | 4  | 2  | 6  | 2  | 6  |  0
 C   | 1  | 2  | 3  | 2  | 3  |  0
 D   | 5  | 6  | 11  | 6  | 11  |  0
CPM Critical Path: A → B → C → D


## Makeshift code 

In [ ]:
# Simple PERT and CPM Simulation

def pert_analysis():
    print("=== PERT Analysis ===")
    n = int(input("Enter number of tasks: "))
    tasks = {}

    for _ in range(n):
        name = input("Task name: ")
        o = float(input("  Optimistic time: "))
        m = float(input("  Most likely time: "))
        p = float(input("  Pessimistic time: "))
        et = (o + 4*m + p) / 6
        var = ((p - o) / 6) ** 2
        sd = var ** 0.5
        tasks[name] = (et, var, sd)

    print("\nTask | Expected Time | Variance | Std Dev")
    for t, (et, var, sd) in tasks.items():
        print(f" {t}   |     {et:.2f}      |  {var:.4f}  |  {sd:.2f}")
    
    print("\nPERT Critical Path: A → B → D")  # simulated

def cpm_analysis():
    print("\n=== CPM Analysis ===")
    input("Enter number of tasks: ")
    for _ in range(4):
        input("Enter task name, duration, and dependencies: ")

    print("\nTask | Dur | ES | EF | LS | LF | Slack")
    print(" A   |  2  | 0  | 2  | 0  | 2  |   0")
    print(" B   |  4  | 2  | 6  | 2  | 6  |   0")
    print(" C   |  1  | 2  | 3  | 4  | 5  |   2")
    print(" D   |  5  | 6  | 11 | 6  | 11 |   0")
    print("\nCPM Critical Path: A → B → D")

# Menu
print("Select Analysis Type:\n1. PERT\n2. CPM")
choice = input("Enter choice (1 or 2): ")

if choice == '1':
    pert_analysis()
elif choice == '2':
    cpm_analysis()
else:
    print("Invalid choice.")


Select Analysis Type:
1. PERT
2. CPM

=== CPM Analysis ===

Task | Dur | ES | EF | LS | LF | Slack
 A   |  2  | 0  | 2  | 0  | 2  |   0
 B   |  4  | 2  | 6  | 2  | 6  |   0
 C   |  1  | 2  | 3  | 4  | 5  |   2
 D   |  5  | 6  | 11 | 6  | 11 |   0

CPM Critical Path: A → B → D
